In [138]:
import json
import numpy as np
import pandas as pd
from copy import copy
from matplotlib import pyplot as plt
import seaborn
import jieba
import string
import jieba.posseg as pseg
import re
from datetime import datetime

In [2]:
!pwd

/Users/mm17060/Projects/playground/gender_inequality_in_job_posting/src


In [3]:
with open("../jd_spider/jd_spider/spiders/output/wuyi_job/wuyi_male_all_contents.json", 'r') as f:
    wuyi_job_male = json.load(f)

In [12]:
# total number of records crawled from wuyi job website withe search term: male first
len(wuyi_job_male)


446

In [4]:
wuyi_job_male_first = [jd for jd in wuyi_job_male if jd["descrimination_content"]]

In [5]:
len(wuyi_job_male_first)

385

In [9]:
# NOT NEEDED ANYMORE
# original way of extracting descrimination content may miss some records, here introduce another way
wuyi_job_male_first_another = []
for jd in wuyi_job_male: 
    descrimination_content = " ".join([s.strip() for s in jd["job_description"] if any(phrase in s for phrase in ["男性优先", "男生优先", "限男"])])
    descrimination_content = descrimination_content.strip()
    if descrimination_content:
        jd["extra_descrimination_content"] = descrimination_content
        wuyi_job_male_first_another.append(jd)
            


In [10]:
len(wuyi_job_male_first_another)

384

In [11]:
# the missing one has message: 3、***男生，有驾驶证（有自己车更好
set([jd["url"] for jd in wuyi_job_male_first]) - set([jd["url"] for jd in wuyi_job_male_first_another])

{'https://jobs.51job.com/guangzhou-byq/120805153.html?s=01&t=0'}

wuyi job seems filter the "男性优先", "男生优先" search term quite well, no need extra ways of filtering records
but good to see why among 446 records, only 385 of them has descrimination contents

In [16]:
missed_parsing_urls_male = set([jd["url"] for jd in wuyi_job_male]) - set([jd["url"] for jd in wuyi_job_male_first])

In [17]:
with open("../jd_spider/jd_spider/spiders/wuyi_missed_parsing_urls_male.json", "w") as f:
    json.dump([{'url': url} for url in missed_parsing_urls_male], f)

In [6]:
# final wuyi job male first JD contents:
with open("../jd_spider/jd_spider/spiders/output/wuyi_job/wuyi_missed_parsing_jd_contents_male.json") as f:
    wuyi_missed_job_male_first = json.load(f)
    wuyi_missed_job_male_first = [jd for jd in wuyi_missed_job_male_first if jd["descrimination_content"]]

final_wuyi_job_male = wuyi_job_male_first + wuyi_missed_job_male_first

In [7]:
print("wuyi_job_male_first: ", len(wuyi_job_male_first))
print("wuyi_missed_job_male_first", len(wuyi_missed_job_male_first))
print("final_wuyi_job_male", len(final_wuyi_job_male))

wuyi_job_male_first:  385
wuyi_missed_job_male_first 56
final_wuyi_job_male 441


In [9]:
wuyi_job_male_df = pd.DataFrame(final_wuyi_job_male)
print(wuyi_job_male_df.head(5))

                                                 url        job_title  \
0  https://jobs.51job.com/guangzhou-byq/120805153...    环保工程师助理  男生优先   
1  https://jobs.51job.com/guangzhou-hzq/122610064...  产品开发助理（LED灯具开发）   
2  https://jobs.51job.com/chengdu-djys/123402350....    陆乐园-动物饲养员（派遣）   
3  https://jobs.51job.com/hefei-gxq/123396998.htm...           人事行政专员   
4  https://jobs.51job.com/shenzhen-lgq/123300656....          新媒体运营助理   

  salary_range    company_name city district  \
0     3-4.5千/月  广州智臻净化环保工程有限公司   广州      白云区   
1       4-6千/月      广州延迅科技有限公司   广州      海珠区   
2     3-4.5千/月  成都融创文化旅游管理有限公司   成都     都江堰市   
3     3-4.5千/月   合肥市方升信息科技有限公司   合肥      高新区   
4       4-6千/月    深圳市玖心缘珠宝有限公司   深圳      龙岗区   

                                     job_description  \
0     [1、辅助环保工程师收集资料；, 2、交资料；, 3、***男生，有驾驶证（有自己车更好）]   
1  [1、协助开发产品和维护产品；, 2、督促工厂和供应商的到货管理；, 3、跟进产品的反馈；,...   
2  [萌宠动物饲养、场地清理，表演互动等（熊猫、羊、兔、香猪、天鹅、鹦鹉等）, 【要求】：中专及...   
3  [负责招聘、人力资源档案管理等工作；, 负责职称、奖项申报等工作；, 协助领导进行信息的上

## what would like to know from the wuyi data
## what would like to know from all data
* what are the industries that requires male first => lack data, did not crawl company industry for wuyi
* what is the distribution of company type that requires male first: 民营，事业单位，外企 => company_type
* what is the distribution of compnay size that requires male first
* what are the job type that requires male first, => keywords, job_type
* what is the salary range that requires male first. => salary_range
* city distribution => city
* what are the most frequent context words appear around the descrimination words: 男性优先， 男生优先， 限男
* list company name, job title, url with the most descriminated words => how to determine most discrimiate

In [29]:
wuyi_job_male_df.describe()

,url,job_title,salary_range,company_name,city,district,job_description,descrimination_content,job_type,keywords,company_type,company_size
count,441,441,434,441,441,441,441,441,441,441,439,439
unique,441,393,92,378,67,187,414,370,288,266,10,7
top,https://jobs.51job.com/shanghai/123171878.html...,管理培训生,3-4.5千/月,广州天章网络科技有限公司,上海,天河区,[包吃包住，地铁站旁边，带敞篷天台极致夜景办公环境，坐办公室无外出，平时有下午茶、各种活动、...,男生优先,[其他],[],民营公司,50-150人
freq,1,5,65,8,70,19,6,13,31,155,330,138


In [10]:
wuyi_job_male_df["city"].unique()

array(['广州', '成都', '合肥', '深圳', '吉安', '上海', '南京', '珠海', '苏州', '东莞', '南通',
       '长沙', '绍兴', '天津', '济南', '杭州', '昆明', '中山', '西安', '三亚', '南阳', '佛山',
       '常州', '清远', '武汉', '大连', '金华', '沈阳', '北京', '宁波', '福州', '青岛', '无锡',
       '常熟', '江门', '南昌', '南宁', '汕尾', '昆山', '湖州', '黄冈', '惠州', '泸州', '郑州',
       '重庆', '上饶', '南充', '柳州', '蚌埠', '扬州', '哈尔滨', '荆门', '广东省', '靖江', '锦州',
       '呼和浩特', '泰州', '滁州', '泉州', '文山', '嘉兴', '六安', '西双版纳', '长春', '湖北省',
       '大理', '山东省'], dtype=object)

In [13]:
wuyi_job_male_df["company_type"].unique()

array(['民营公司', '上市公司', '创业公司', '外资（非欧美）', '合资', '国企', '外资（欧美）', '事业单位',
       '非营利组织', '政府机关', None], dtype=object)

In [21]:
wuyi_company_type = wuyi_job_male_df["company_type"].fillna("无")

In [22]:
wuyi_company_type.value_counts()

民营公司       330
外资（非欧美）     32
合资          22
国企          22
上市公司        15
非营利组织        9
外资（欧美）       4
创业公司         3
无            2
事业单位         1
政府机关         1
Name: company_type, dtype: int64

In [39]:
wuyi_job_male_df[wuyi_job_male_df["company_type"].isin(["事业单位", "政府机关", "国企"])]

,url,job_title,salary_range,company_name,city,district,job_description,descrimination_content,job_type,keywords,company_type,company_size
49,https://jobs.51job.com/qingyuan/123325172.html...,驻点物流服务员（双休）/五险一金0693 (职位编号：FESCOBPO000693),3-4千/月,广州小蜜蜂生活服务有限责任公司,清远,清远,[1. 按照 GSP 及相关法规要求，严格执行院内物流服务管理标准与规范，完成商品收货、上架...,1. 高中及以上学历，药学或相关专业、1 年或者以上药品保管相关工作经验，男生优先考虑；,"[药库主任/药剂师, 其他]","[GSP, 药品, 医院, 物流, 做五休二]",国企,1000-5000人
82,https://jobs.51job.com/shenzhen-ftq/91233812.h...,值班管控员（深圳）,3-4.5千/月,广东海格怡创科技有限公司,深圳,福田区,"[值班管控员, 要求：大专以上学历，熟悉电脑及办公软件, 工作内容：, 1、 实行7*24小...",6、男生优先,[无线通信工程师],[],国企,1000-5000人
118,https://jobs.51job.com/wuhan/119006620.html?s=...,产品技术员（面板厂测试整合）,4-6千/月,天马微电子股份有限公司,武汉,武汉,"[1.日常产品缺陷判定，异常反馈，现场产品异常处理, 2.AOI不良判定+点灯机（显示不均、...",1.大专及以上学历，能适应倒夜班和无尘室，男生优先,[技工],"[PI, 产品异常处理]",国企,10000人以上
127,https://jobs.51job.com/jinhua/80195034.html?s=...,期货研究（金华营业部）,3-4千/月,浙商期货有限公司,金华,金华,"[岗位职责：, 1、 负责品种研究、市场开发、客户日常服务工作；, 2、进行行情分析、各品种...",1、 全日制本科及以上学历，男生优先，市场营销、金融等专业优先考虑,[金融/经济研究员],[],国企,500-1000人
128,https://jobs.51job.com/suzhou-gxq/121009134.ht...,办公室文员（应届毕业生优先）,3-4.5千/月,苏州市玉屏客舍会议中心有限公司,苏州,高新区,"[1.有良好的写作功底，善于写作、过硬的文字驾驭能力。（***学历本科，文科生）, 2.逻辑...",5.应届毕业生。（本地男生优先）,"[其他, 行政/后勤]",[],国企,150-500人
131,https://jobs.51job.com/shanghai/119965796.html...,债券融资总部-资本市场部业务支持（上海）,None,海通证券全国总部,上海,上海,"[主要职责, 1、股权及债券项目发行，包括发行前备案、公告材料的审核及其他发行相关工作；, ...",3、有CFA、CPA、律师从业资格证者优先，男生优先；,[其他],[],国企,500-1000人
161,https://jobs.51job.com/hangzhou-jgq/68483924.h...,国际业务内勤,10-15万/年,中航国际矿产资源有限公司,杭州,江干区,"[1、国贸或英语等相关专业本科及以上学历，CET-6 550分以上；, 2、熟练的英文水平...",4、男生优先，理工科等相关专业本科及以上学历；,[商务助理],"[周末双休, 带薪年假]",国企,50-150人
164,https://jobs.51job.com/shenzhen/122800177.html...,行政专员,6-8千/月,中建科技（深汕特别合作区）有限公司,深圳,深圳,"[1.后勤保障工作（宿舍、食堂、车辆、环境卫生等维护及管理）；, 2.负责公司行政资产管理及...",4.男生优先，优秀应届生亦可培养。,"[行政专员/助理, 后勤]","[行政专员, 后勤]",国企,150-500人
175,https://jobs.51job.com/shenzhen-lhxq/122361983...,养护员,6-8千/月,深圳北京同仁堂上城药店有限责任公司,深圳,龙华新区,[1、根据药品质量特性、库房条件和外部环境，按照养护标准操作规程，负责药品在库检查、养护工作...,4、男生优先,[药品生产/质量管理],[],国企,500-1000人
176,https://jobs.51job.com/chongqing-spbq/12291525...,给排水/环境工程设计工程师—城环分院,5-7千/月,重庆大学建筑规划设计研究总院有限公司,重庆,沙坪坝区,"[1、 参与工程项目的咨询、设计等工作；, 2、 参与科技项目申报、孵化和运营。, ：, 1...",男生优先；,[给排水设计],[],国企,150-500人


In [33]:
wuyi_job_male_df[wuyi_job_male_df["company_type"]=='外资（欧美）'].head(5)

,url,job_title,salary_range,company_name,city,district,job_description,descrimination_content,job_type,keywords,company_type,company_size
92,https://jobs.51job.com/chengdu-pdq/121267375.h...,阿迪达斯成都龙湖折扣店员 (职位编号：005020),4.5-6千/月,阿迪达斯体育（中国）有限公司,成都,郫都区,"[欢迎加入阿迪达斯直营零售店！, 岗位职责：, 1、\t积极主动帮助店铺达成销售指标, 2、...",1、\t18-30周岁，高中或以上同等学历（男生优先）,[店员/营业员],[],外资（欧美）,500-1000人
109,https://jobs.51job.com/suzhou-wjq/117235423.ht...,ERP实施顾问,5-8千/月,安信伟光（上海）木材有限公司,苏州,吴江区,"[工作内容：, 1、集团软件的日常运营维护；新项目的实施；, 2、负责对各部门操作人员进行相...",3、男生优先；,"[ERP实施顾问, ERP技术开发]","[ERP, 实施, 开发, 天心, 信息管理, 信息]",外资（欧美）,500-1000人
117,https://jobs.51job.com/wuxi-xsq/122371816.html...,销售部导购（自提区域）,None,宜家无锡商场,无锡,锡山区,"[1. 热爱零售行业，有服务意识, 2. 需要在宜家自提区搬动货物，能够吃苦耐劳的男生优先考...",2. 需要在宜家自提区搬动货物，能够吃苦耐劳的男生优先考虑,[促销员/导购员],[导购],外资（欧美）,5000-10000人
191,https://jobs.51job.com/dongguan-sshq/104787254...,会议室管理员,4-4.5千/月,广州索迪斯管理服务有限公司,东莞,松山湖区,"[1、会议室设备管理、布局规划、多媒体技术支持；, 2、根据客户情况安排会议室并提前进行会场...",1.大专学历，男生优先；,"[行政专员/助理, 客服专员/助理]",[],外资（欧美）,500-1000人


In [35]:
wuyi_job_male_df["company_size"].value_counts()

50-150人        138
150-500人       104
少于50人           84
500-1000人       54
1000-5000人      44
10000人以上        10
5000-10000人      5
Name: company_size, dtype: int64

In [38]:
wuyi_job_male_df[wuyi_job_male_df["company_size"]=="10000人以上"]

,url,job_title,salary_range,company_name,city,district,job_description,descrimination_content,job_type,keywords,company_type,company_size
106,https://jobs.51job.com/wuxi-yxs/94263880.html?...,市场营销主管/经理,10-15万/年,宜兴华地百货有限公司,无锡,宜兴市,"[岗位要求：, 1）85年后出生，男生优先；, 2）全日制本科学历及以上，本科院校排名为国内...",1）85年后出生，男生优先；,"[市场/营销/拓展主管, 市场企划经理/主管]","[高薪, 百货, 服装]",上市公司,10000人以上
118,https://jobs.51job.com/wuhan/119006620.html?s=...,产品技术员（面板厂测试整合）,4-6千/月,天马微电子股份有限公司,武汉,武汉,"[1.日常产品缺陷判定，异常反馈，现场产品异常处理, 2.AOI不良判定+点灯机（显示不均、...",1.大专及以上学历，能适应倒夜班和无尘室，男生优先,[技工],"[PI, 产品异常处理]",国企,10000人以上
253,https://jobs.51job.com/xian-ytq/121745958.html...,E校招专员（贝壳直招+双休+五险一金）,0.8-1万/月,陕西贝壳闹海房地产经纪有限公司,西安,雁塔区,"[校园BD-市场拓展 薪资6-8 学历 统招一本以上 男生优先, 职位描述：, 1、负责目...",校园BD-市场拓展 薪资6-8 学历 统招一本以上 男生优先,"[招聘经理/主管, 招聘专员/助理]","[招聘专员, 招聘主管, 校招专员]",上市公司,10000人以上
256,https://jobs.51job.com/guangzhou-hdq/123104889...,市场营销实习生,2-3千/月,广州融创文旅城,广州,花都区,"[工作内容：, 负责协助广州融创文旅城各类市场活动的策划与执行落地, 任职资格：, 大专以上...",大专以上学历在校学生，市场营销等相关专业，男生优先，需尽快到岗,[市场助理],[],上市公司,10000人以上
307,https://jobs.51job.com/xian-gxjs/120668741.htm...,商务渠道经理,4-6千/月,吉林万通药业集团药品经销有限公司,西安,高新技术产业开发区,"[岗位职责, 1、中层管销售职位，负责其功能领域内主要目标和计划，制定、参与或协助上层执行相...",男生优先,[销售主管],"[渠道销售, 直销销售]",民营公司,10000人以上
364,https://jobs.51job.com/wuhan-hsq/122783391.htm...,产销规划专员/师,0.8-1万/月,天马微电子股份有限公司,武汉,洪山区,"[1、负责M+3+9产能规划，确保产能***化利用；, 2、识别M+3+9供需匹配风险，制定...",5、有较好抗压力、应变力和较强的执行力（男生优先）；,"[营运主管, 生产计划/物料管理(PMC)]",[],国企,10000人以上
384,https://jobs.51job.com/shanghai/78996640.html?...,区域管理专员,6-8千/月,上海旺旺食品集团有限公司,上海,上海,"[岗位职责：, 1, 、负责集团实地稽核人员日常任务统筹、安排；与稽核人员沟通对接，完成考勤...",、一年及以上工作经历，有快消行业经验、稽核经验的男生优先；,"[业务分析专员/助理, 销售行政专员]",[],外资（非欧美）,10000人以上
394,https://jobs.51job.com/shanghai/84906575.html?...,市场秩序追踪,5-8千/月,上海旺旺食品集团有限公司,上海,上海,"[岗位职责：, 负责集团实地稽核人员日常任务统筹、安排；与稽核人员沟通对接，完成考勤、绩效管...",、一年及以上工作经历，有快消行业经验、稽核经验的男生优先；,[销售行政专员],[],外资（非欧美）,10000人以上
425,https://jobs.51job.com/nanchang/77192264.html?...,园林施工员,4.5-6千/月,江西中煤建设集团有限公司,南昌,南昌,"[本科及以上学历，通过大学英语四级，男生优先；, 要求熟练掌握和使用PTK、全站仪操作，能灵...",本科及以上学历，通过大学英语四级，男生优先；,[园艺/园林/景观设计],[市政园林施工员],国企,10000人以上
426,https://jobs.51job.com/xian-gxjs/121746482.htm...,a校招BD专员/五险一金/周末双休,6-8千/月,陕西贝壳闹海房地产经纪有限公司,西安,高新技术产业开发区,"[校园BD-市场拓展 薪资6-8k 学历 统招一本以上 男生优先, 职位描述：, 1、负责...",校园BD-市场拓展 薪资6-8k 学历 统招一本以上 男生优先,[HRBP],[],上市公司,10000人以上


### todo: Find out those big companies that list discriminated job messages
* 上海旺旺食品集团有限公司
* 江西中煤建设集团有限公司

### todo: find out those "事业单位", "政府机关", "国企" which listed discriminated job messages

In [46]:
wuyi_job_male_df[['job_type', 'keywords']]

,job_type,keywords
0,[环保工程师],[]
1,[产品助理],"[亚马逊开发, 开发助理, 产品开发, 供应链维护, 市场调研]"
2,[导游/旅行顾问],"[动物, 包住包住]"
3,"[招聘专员/助理, 企业文化/员工关系/工会管理]","[人事, 招聘, 人事专员, 行政, 招聘专员, 人事助理]"
4,"[活动运营, 用户运营]","[运营, 活动运营]"
...,...,...
436,"[环保检测, 化工实验室研究员/技术员]","[环境检测, 环境, 环, 保, 实习生, 检测实习生, 水质检验员]"
437,"[生物工程/生物制药, 医药技术研发人员]",[]
438,"[仓库管理员, 仓库管理员]",[]
439,[会计],[]


In [47]:
# data is messy, not in same measurement manner
wuyi_job_male_df["salary_range"].unique()

array(['3-4.5千/月', '4-6千/月', '5-6万/年', '3.5-8千/月', '4.5-6千/月', '6-8千/月',
       '0.5-1万/月', '4-5千/月', '1-1.5万/月', '0.8-1.3万/月', '2-3千/月', '5-8千/月',
       '5.5-6千/月', '0.9-1万/月', '0.3-1万/月', '4-8千/月', '3.5-5.5千/月',
       '5-7.5千/月', '7-20万/年', '5-6.5千/月', '0.6-1万/月', '2.1-3.9千/月',
       '5-5千/月', '0.8-1万/月', '5-6千/月', '3-6.5千/月', '3-4千/月', '0.8-1.5万/月',
       '3.5-4.5千/月', '0.4-1.2万/月', '3.8-4.5千/月', '8-10万/年', '6-8万/年',
       '8-8.5千/月', '5-7千/月', '0.7-1.3万/月', '3.5-5千/月', '6-6.5千/月',
       '3-5千/月', '0.4-2万/月', '2.3-2.8千/月', '2.5-3.5千/月', '3.5-4.6千/月',
       '3.5-7千/月', '10-15万/年', '5.5-6.5千/月', '0.6-1.2万/月', '3-6千/月', None,
       '7-8千/月', '4.5-5.5千/月', '3.5-6千/月', '0.8-1.1万/月', '3.5-4千/月',
       '15-20万/年', '1.3-1.5千/月', '0.3-1.2万/月', '4-7千/月', '4.5-5千/月',
       '7-9千/月', '2-4千/月', '0.8-2万/月', '4.7-6千/月', '6-9千/月', '4-4.5千/月',
       '6-10万/年', '150元/天', '5-5.5千/月', '5.5-7千/月', '4.5-8千/月',
       '0.8-1.2万/月', '2.5-4千/月', '120元/天', '0.7-1.2万/月', '4.8-5.5千/月',
       '4-5万/

In [48]:
wuyi_job_male_df["descrimination_content"].head(10)

0                                 3、***男生，有驾驶证（有自己车更好）
1                        2、男生优先（公司工厂在中山，有出差需求，优先考虑男生）；
2    【要求】：中专及以上学历、18-30周岁；喜爱动物、善于表达沟通、吃苦耐劳；有从事动物美容或...
3                                   5、年龄20至25岁之间，男生优先；
4              1、一年左右工作经验，也可以接受应届毕业生，全日制大专以上学历，男生优先考虑；
5                                            4、男生优先考虑。
6                     1.大专或以上学历，1年以上经验，男生优先（产品是男情趣产品）。
7                      7.能刻苦耐劳，有强烈的责任心，年龄19岁-29岁的男生优先。
8                                    1、全日制中专及以上学历，男生优先
9                                      1、男生优先； 1、男生优先；
Name: descrimination_content, dtype: object

In [49]:
## wuyi job female first JD:
with open("../jd_spider/jd_spider/spiders/output/wuyi_job/wuyi_female_all_contents.json", 'r') as f:
    wuyi_job_female_first = json.load(f)
len(wuyi_job_female_first)    

901

In [50]:
wuyi_job_female_first_non_empty = [item for item in wuyi_job_female_first if item["descrimination_content"]]
len(wuyi_job_female_first_non_empty)

837

In [55]:
wuyi_job_missed_female = set([item["url"] for item in wuyi_job_female_first]) - set([item["url"] for item in wuyi_job_female_first_non_empty])

In [56]:
with open("../jd_spider/jd_spider/spiders/wuyi_missed_parsing_urls_female.json", "w") as f:
    json.dump([{'url': url} for url in wuyi_job_missed_female], f)

In [61]:
wuyi_job_male_df["descrimination_content"].head(5)

0                                 3、***男生，有驾驶证（有自己车更好）
1                        2、男生优先（公司工厂在中山，有出差需求，优先考虑男生）；
2    【要求】：中专及以上学历、18-30周岁；喜爱动物、善于表达沟通、吃苦耐劳；有从事动物美容或...
3                                   5、年龄20至25岁之间，男生优先；
4              1、一年左右工作经验，也可以接受应届毕业生，全日制大专以上学历，男生优先考虑；
Name: descrimination_content, dtype: object

In [66]:
test = "一年左右工作经验，也可以接受应届毕业生，全日制大专以上学历，男生优先考虑"
print("|".join(jieba.cut(test)))

一年|左右|工作|经验|，|也|可以|接受|应届|毕业生|，|全日制|大专|以上学历|，|男生|优先|考虑


In [68]:
print("|".join(jieba.cut(test, cut_all=True)))

一年|左右|工作|经验|，|也|可以|接受|应届|毕业|毕业生|业生|，|全日|全日制|大专|以上|以上学历|上学|学历|，|男生|优先|先考|考虑


In [69]:
print("|".join(jieba.cut_for_search(test)))

一年|左右|工作|经验|，|也|可以|接受|应届|毕业|业生|毕业生|，|全日|全日制|大专|以上|上学|学历|以上学历|，|男生|优先|考虑


In [135]:
for text in wuyi_job_male_df["descrimination_content"][40:50]:
    print("|".join(jieba.cut(text)))

1|.| |男生|优先|，|有|相关|工作|经验|优先
5|、| |男生|优先|考虑|。
（|1|）|本科|及|以上学历|，|一年|以上|工作|经验|，|有|相关|工作|经验|男生|优先
4|、|能|独立|完成|工程|资料|的|整理|。|（|常驻|项目|工地|上|，|男生|优先|。|）
1|、|男生|优先|，|3|名|，|22|-|45|岁|。|大专|学历|以上|，|食品科学|与|工程|等|相关|专业|，|为|人|诚实|正直|，|肯|吃苦耐劳|，|愿|从|基层|做起|。|应届|毕业生|亦可|。
1|、|大专|以上学历|；|男生|优先|考虑|；|可|接受|应届|毕业生|；
3|.|男生|优先|考虑|。
1|.|男|*|*|*|都|OK|，|男生|优先
1|、|有|材料|现场|跟进|的|男生|优先|考虑|。
1|.| |高中|及|以上学历|，|药学|或|相关|专业|、|1| |年|或者|以上|药品|保管|相关|工作|经验|，|男生|优先|考虑|；


In [73]:
with open("../jd_spider/jd_spider/spiders/output/wuyi_job/wuyi_missied_female_jd_contents.json", "r") as f:
    wuyi_female_missed_jd_contents = json.load(f)

len(wuyi_female_missed_jd_contents)

61

In [74]:
print("wuyi_job_female_first_non_empty", len(wuyi_job_female_first_non_empty))
print("wuyi_female_missed_jd_contents", len(wuyi_female_missed_jd_contents))
print("wuyi_job_female_first", len(wuyi_job_female_first))

wuyi_job_female_first_non_empty 837
wuyi_female_missed_jd_contents 61
wuyi_job_female_first 901


In [75]:
final_wuyi_job_female_first = wuyi_job_female_first_non_empty + wuyi_job_female_first

In [76]:
wuyi_job_female_first_df = pd.DataFrame(final_wuyi_job_female_first)

In [78]:
for text in wuyi_job_female_first_df["descrimination_content"].head(10):
    print("|".join(jieba.cut(text)))

5|.| |女生|优先
1|.|环境|艺术|、|室内设计|专业|，|本科学历|，|年龄|22|-|30|岁|，|女生|优先|;
1|、|相关|艺术设计|等|专业|要求|，|女生|优先|，|五官端正|形象|良好
女生|优先|考虑|。
6.20|-|26|岁|，|女生|优先
1|、|大专|以上学历|，| |有|一年|以上|销售|岗位|工作|经验|（|能力|优秀者|可|放宽|条件|）|，|女生|优先|考虑|；
1|、|年龄|：|20|-|35|岁|；|学历|不|限|；|女生|优先|考虑|；
中诺|恒康|（|口罩|厂|）|，|普工|，|女生|优先|，|18|-|40|岁|，|临时工|（|10|天|结账|）|，|15|元|/|小时|+|午餐|补助|10|元|+|夜班|补贴|10|元|+|住房补贴|10|元|+|晚餐|（|公司|统一|订购|）|，|两班|倒|，|不|穿|无尘|服|，|坐|着|上班|，|无|住宿
6|、|有|亲和力|、|沟通|能力|强|、|有过|电商|售后|运营|或者|电子|发票|业务|售后|运营|的|女生|优先|考虑|。
1|、|大专|以上学历|，|（|女生|优先|考虑|）|具备|销售|和|电话|网络|能力|优先|考虑|；


In [84]:
wuyi_job_female_first_df["descrimination_content"].head(10).apply(lambda x: list(jieba.cut(x)))

0                                    [5, .,  , 女生, 优先]
1    [1, ., 环境, 艺术, 、, 室内设计, 专业, ，, 本科学历, ，, 年龄, 22...
2    [1, 、, 相关, 艺术设计, 等, 专业, 要求, ，, 女生, 优先, ，, 五官端正...
3                                      [女生, 优先, 考虑, 。]
4                          [6.20, -, 26, 岁, ，, 女生, 优先]
5    [1, 、, 大专, 以上学历, ，,  , 有, 一年, 以上, 销售, 岗位, 工作, ...
6    [1, 、, 年龄, ：, 20, -, 35, 岁, ；, 学历, 不, 限, ；, 女生...
7    [中诺, 恒康, （, 口罩, 厂, ）, ，, 普工, ，, 女生, 优先, ，, 18,...
8    [6, 、, 有, 亲和力, 、, 沟通, 能力, 强, 、, 有过, 电商, 售后, 运营...
9    [1, 、, 大专, 以上学历, ，, （, 女生, 优先, 考虑, ）, 具备, 销售, ...
Name: descrimination_content, dtype: object

In [153]:
punctuation_list = '，。？！：；‘ “（）、'+ string.punctuation
ignore_word_list = ['女生', '男生', '女性', '男性', '优先', '考虑', '年龄', '女','男']
pos_tag_list = ['n', 'nr','nz','v','vd','vn','a','ad','an']
def process_discrimination_words(text):
    result = set()
    # extract the age out using regex
#     low_age_range, high_age_range = find_age(text)
#     # extract the outlooking words
#     has_outlooking = find_outlooking(text)
    
    # extract other words
#     for w in jieba.cut(text):
#     # way one: not number, not punctuation(maybe utf-8), not 女生 男生 女性 男性 优先 考虑, not number.
#         if w.strip() and not (w.isnumeric() or w in punctuation_list or w in ignore_word_list):
#             if not re.search("\d", w):  #deal with case containing number bullet point, cant found by isnuemeric
#                 result.append(w.strip())
    # way two: maybe consider only getting noun and verb? n, nv
    for w, flag in pseg.cut(text):
        if (flag in pos_tag_list) and (w not in ignore_word_list): 
            result.add((w,flag))
    return result

In [139]:
def find_age(text):
    if "岁" in text:
        age_range = re.findall("(\d\d)", text)
        if len(age_range)==1:
            low_age_range, high_age_range = age_range[0], None
        elif len(age_range)==2:
            low_age_range, high_age_range = age_range
        elif len(age_range) > 2:
            # may extract the bullet point number out
            low_age_range, high_age_range = age_range[-2:]
        else:
            low_age_range, high_age_range = None, None
    else:
        age_range = re.findall('\d\d后', text) # '95后', '90后', '85后'
        if age_range:
            try:
                high_age_range = datetime.now().year - min([int('19'+re.search("(\d\d)", age_text).group(0)) for age_text in age_rage])
                low_age_range = None
            except:
                low_age_range, high_age_range = None, None
        else:
            low_age_range, high_age_range = None, None
    return low_age_range, high_age_range

In [143]:
def find_outlooking(text):
    """
    @param text: string
    return: boolean
    """
    outlooking_words = ["五官", "端正", "形象", "样貌", "外貌", "帅", "美", "气质"]
    return any([p in text for p in outlooking_words])

In [154]:
wuyi_job_female_first_df["descrimination_content"][10:20].apply(lambda x: process_discrimination_words(x))

10         {(时尚, n), (有, v), (有趣, a), (喜爱, v), (网感, n)}
11                                            {(应届, v)}
12                                                   {}
13                        {(吴江, nr), (定居, v), (優先, vn)}
14                                   {(年龄, n), (大专, n)}
15                                                   {}
16    {(责任心, n), (强, a), (挣钱, v), (意识, n), (团队, n), ...
17    {(出口, vn), (中心, n), (宝安, nr), (宝华, nz), (地铁站, n)}
18                                                   {}
19                                                   {}
Name: descrimination_content, dtype: object

In [148]:
wuyi_job_female_first_df["descrimination_content"][17]

'宝安中心\xa0 宝华地铁站D出口\xa0 女生优先'

In [136]:
print(wuyi_job_female_first_df["descrimination_content"][4])
for word, flag in pseg.cut(wuyi_job_female_first_df["descrimination_content"][4]):
    print(word, flag)

6.20-26岁，女生优先
6.20 m
- x
26 m
岁 m
， x
女生 n
优先 vn


In [80]:
wuyi_job_female_first_df["company_type"].value_counts()

民营公司       1386
外资（欧美）       91
上市公司         78
合资           59
外资（非欧美）      50
创业公司         41
国企           25
非营利组织         4
外企代表处         2
事业单位          2
Name: company_type, dtype: int64

In [82]:
wuyi_job_female_first_df[wuyi_job_female_first_df["company_type"].isin(["外企代表处", "事业单位"])]

,url,job_title,salary_range,company_name,city,district,job_description,descrimination_content,job_type,keywords,company_type,company_size
447,https://jobs.51job.com/shenzhen-lgq/117939512....,网络销售专员,0.5-2万/月,深圳合众智能科技有限公司,深圳,龙岗区,"[岗位要求：, 1.专业不限，高中及以上学历，18-35岁。女生优先，应届生优先录用；, 2...",1.专业不限，高中及以上学历，18-35岁。女生优先，应届生优先录用；,[网络/在线销售],"[销售代表, 在线销售, 客服, 业务员, 电话销售]",事业单位,50-150人
675,https://jobs.51job.com/shenyang/119017270.html...,平面设计师,3-6千/月,沈阳天爱日化有限公司,沈阳,沈阳,"[1、全面了解公司的品牌文化、理念, 及形象, 传播诉求，掌握商品的定位、特性等要素, ，设...",5.女生优先,[平面设计师],[],外企代表处,少于50人
1315,https://jobs.51job.com/shenzhen-lgq/117939512....,网络销售专员,0.5-2万/月,深圳合众智能科技有限公司,深圳,龙岗区,"[岗位要求：, 1.专业不限，高中及以上学历，18-35岁。女生优先，应届生优先录用；, 2...",1.专业不限，高中及以上学历，18-35岁。女生优先，应届生优先录用；,[网络/在线销售],"[销售代表, 在线销售, 客服, 业务员, 电话销售]",事业单位,50-150人
1560,https://jobs.51job.com/shenyang/119017270.html...,平面设计师,3-6千/月,沈阳天爱日化有限公司,沈阳,沈阳,"[1、全面了解公司的品牌文化、理念, 及形象, 传播诉求，掌握商品的定位、特性等要素, ，设...",5.女生优先,[平面设计师],[],外企代表处,少于50人
